## Import packages

In [1]:
%load_ext autoreload
%autoreload 2

import sys
import numpy as np
import seaborn as sns
import pandas as pd
import pickle
import scanpy as sc
sc.set_figure_params(dpi=100, dpi_save=300)
import scvi
import anndata as ad
from matplotlib import pyplot, cm
import os
from math import ceil
from scipy.stats import spearmanr
import math
from time import time

import leidenalg

from anndata import AnnData
import scanpy as sc
from scanpy import read
import pandas as pd

import matplotlib.pyplot as plt
from sciPENN.Preprocessing import preprocess

from os.path import join
import shutil

print(scvi.__version__)

0.9.1


# Read data: Pbmc (train), H1N1 (test)

In [2]:
adata_gene = sc.read("../Data/pbmc/pbmc_gene.h5ad")
adata_protein = sc.read("../Data/pbmc/pbmc_protein.h5ad")

In [3]:
adata_gene_test = sc.read("../Data/H1N1/gene_data.mtx").T
adata_gene_test.var.index = pd.read_csv("../Data/H1N1/gene_names.txt", index_col = 0).iloc[:, 0]
adata_gene_test.obs = pd.read_csv("../Data/H1N1/meta_data.txt", sep = ',', index_col = 0)

In [4]:
adata_protein_test = sc.read("../Data/H1N1/protein_data.mtx").T
adata_protein_test.var.index = [x[:len(x) - 5] for x in pd.read_csv("../Data/H1N1/protein_names.txt", index_col = 0).iloc[:,0]]
adata_protein_test.obs = pd.read_csv("../Data/H1N1/meta_data.txt", sep = ',', index_col = 0)

adata_protein_test.layers["raw"] = adata_protein_test.X

In [5]:
base = "runtime"
train_path, test_path = join(base, 'train'), join(base, 'test')

indices, indices_test = {}, {}
for path in os.listdir(train_path):
    indices[float(path[3:6])] = pd.read_csv(join(train_path, path))['idx'].tolist()
    indices_test[float(path[3:6])] = pd.read_csv(join(test_path, path))['idx'].tolist()

In [7]:
times = {}

for frac in sorted(indices):
    idx, idx_test = indices[frac], indices_test[frac]
    gene_train, protein_train, gene_test, bools, train_keys, categories = preprocess([adata_gene[idx]], [adata_protein[idx]], adata_gene_test[idx_test], train_batchkeys = ["donor"], test_batchkey = "sample", gene_list = [], select_hvg = True, cell_normalize = True, log_normalize = True, gene_normalize = True, min_cells = 30, min_genes = 200)
    
    hvg = gene_test.var_names
    cells_test = gene_test.obs_names
    cells_train = gene_train.obs_names
    
    adata_gene_pbmc_hvg = adata_gene[idx][cells_train, hvg].copy()
    adata_gene_h1n1_hvg = adata_gene_test[idx_test][cells_test, hvg].copy()
    
    adata_protein_tmp = adata_protein[idx][cells_train, :].copy()
    adata_protein_tmp_test = adata_protein_test[idx_test][cells_test, :].copy()
    
    adata_gene_pbmc_hvg.obs['patient'] = pd.DataFrame(adata_gene_pbmc_hvg.obs['donor']).copy()
    adata_gene_pbmc_hvg.obs['patient'] = adata_gene_pbmc_hvg.obs['donor'].astype("str")
    
    adata_gene_h1n1_hvg.obs['patient'] = pd.DataFrame(adata_gene_h1n1_hvg.obs['sample']).copy()
    adata_gene_h1n1_hvg.obs['patient'] = adata_gene_h1n1_hvg.obs['patient'].astype("str")
    
    adata = ad.concat([adata_gene_pbmc_hvg.copy(), adata_gene_h1n1_hvg.copy()],
                     join='outer')
    
    adata_final = adata.copy()
    
    n, p = adata_protein_tmp.shape
    n_H1N1, p_H1N1 = adata_protein_tmp_test.shape

    protein_dat = pd.DataFrame(np.zeros(shape = (n + n_H1N1, p), dtype = 'float32'), 
                               index = list(adata_protein_tmp.obs_names) + list(adata_protein_tmp_test.obs_names),
                               columns = adata_protein_tmp.var_names)

    protein_dat.iloc[:n] = adata_protein_tmp.X.toarray().copy() #fill the protein training data, leave test data as 0s

    adata_final.obsm["protein_expression"] = protein_dat
    
    start = time()
    
    scvi.data.setup_anndata(adata_final, batch_key="patient", 
                        protein_expression_obsm_key="protein_expression")
    
    totalvae = scvi.model.TOTALVI(
    adata_final,
    latent_distribution = "normal",
    n_layers_decoder = 2)
    
    totalvae.train(400, 4e-4)
        
    _, protein_means = totalvae.get_normalized_expression(
        transform_batch = adata_gene_h1n1_hvg.obs["patient"].unique()[0:8],
        include_protein_background=True,
        sample_protein_mixing=False,
        return_mean=True,
    )
    
    times[frac] = time() - start

Trying to set attribute `.obs` of view, copying.
Trying to set attribute `.obs` of view, copying.
Trying to set attribute `.obs` of view, copying.



QC Filtering Training Cells
QC Filtering Testing Cells

QC Filtering Training Genes
QC Filtering Testing Genes

Normalizing Training Cells
Normalizing Testing Cells

Log-Normalizing Training Data
Log-Normalizing Testing Data

Finding HVGs


... storing 'orig.ident' as categorical
... storing 'lane' as categorical
... storing 'donor' as categorical
... storing 'time' as categorical
... storing 'celltype.l1' as categorical
... storing 'celltype.l2' as categorical
... storing 'celltype.l3' as categorical
... storing 'Phase' as categorical
... storing 'batch' as categorical
... storing 'Dataset' as categorical
... storing 'barcode_check' as categorical
... storing 'tenx_lane' as categorical
... storing 'cohort' as categorical
... storing 'hash_maxID' as categorical
... storing 'hash_secondID' as categorical
... storing 'hto_classification' as categorical
... storing 'hto_classification_global' as categorical
... storing 'hash_ID' as categorical
... storing 'adjmfc.time' as categorical
... storing 'DMX_GLOBAL_BEST' as categorical
... storing 'DEMUXLET.BARCODE' as categorical
... storing 'sample' as categorical
... storing 'joint_classification_global' as categorical
... storing 'timepoint' as categorical
... storing 'K0' as ca


Normalizing Gene Training Data by Batch


100%|██████████| 8/8 [00:00<00:00, 10.39it/s]


...


Normalizing Gene Testing Data by Batch


100%|██████████| 20/20 [00:01<00:00, 16.86it/s]


INFO     Using batches from adata.obs["patient"]                                             
INFO     No label_key inputted, assuming all cells have same label                           
INFO     Using data from adata.X                                                             
INFO     Computing library size prior per batch                                              
INFO     Using protein expression from adata.obsm['protein_expression']                      
INFO     Using protein names from columns of adata.obsm['protein_expression']                
INFO     Found batches with missing protein expression                                       
INFO     Successfully registered anndata object containing 214948 cells, 1000 vars, 28       
         batches, 1 labels, and 224 proteins. Also registered 0 extra categorical covariates 
         and 0 extra continuous covariates.                                                  
INFO     Please do not further modify adata until model is t

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 318/400:  80%|███████▉  | 318/400 [3:54:42<1:00:31, 44.29s/it, loss=390, v_num=1]


In [8]:
for key in times:
    times[key] = [times[key]]

/home/jlakkis/miniconda3/envs/scipen/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [9]:
pd.DataFrame(times, index = ['totalVI']).T.to_csv(join(base, "totalVI.csv"))